根据增益规则  手写决策树

In [36]:
'''数据导入，互联网上随便复制的一则数据'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import  tree
str1 = "Day	Outlook	Temp.	Humidity	Wind	Decision \
1	Sunny	Hot	High	Weak	No \
2	Sunny	Hot	High	Strong	No \
3	Overcast	Hot	High	Weak	Yes \
4	Rain	Mild	High	Weak	Yes \
5	Rain	Cool	Normal	Weak	Yes \
6	Rain	Cool	Normal	Strong	No \
7	Overcast	Cool	Normal	Strong	Yes \
8	Sunny	Mild	High	Weak	No \
9	Sunny	Cool	Normal	Weak	Yes \
10	Rain	Mild	Normal	Weak	Yes \
11	Sunny	Mild	Normal	Strong	Yes \
12	Overcast	Mild	High	Strong	Yes \
13	Overcast	Hot	Normal	Weak	Yes \
14	Rain	Mild	High	Strong	No "
data = str1.split()
data = np.array(data).reshape((6,15))
data = data.tolist()
data

[['Day',
  'Outlook',
  'Temp.',
  'Humidity',
  'Wind',
  'Decision',
  '1',
  'Sunny',
  'Hot',
  'High',
  'Weak',
  'No',
  '2',
  'Sunny',
  'Hot'],
 ['High',
  'Strong',
  'No',
  '3',
  'Overcast',
  'Hot',
  'High',
  'Weak',
  'Yes',
  '4',
  'Rain',
  'Mild',
  'High',
  'Weak',
  'Yes'],
 ['5',
  'Rain',
  'Cool',
  'Normal',
  'Weak',
  'Yes',
  '6',
  'Rain',
  'Cool',
  'Normal',
  'Strong',
  'No',
  '7',
  'Overcast',
  'Cool'],
 ['Normal',
  'Strong',
  'Yes',
  '8',
  'Sunny',
  'Mild',
  'High',
  'Weak',
  'No',
  '9',
  'Sunny',
  'Cool',
  'Normal',
  'Weak',
  'Yes'],
 ['10',
  'Rain',
  'Mild',
  'Normal',
  'Weak',
  'Yes',
  '11',
  'Sunny',
  'Mild',
  'Normal',
  'Strong',
  'Yes',
  '12',
  'Overcast',
  'Mild'],
 ['High',
  'Strong',
  'Yes',
  '13',
  'Overcast',
  'Hot',
  'Normal',
  'Weak',
  'Yes',
  '14',
  'Rain',
  'Mild',
  'High',
  'Strong',
  'No']]

采用树结构：二叉树
选择属性准则：信息增益
目的：分类


算法思想

生成决策树函数（训练样本）
if 训练样本为空 ： 返回一个空结点
end if 训练样本信息增益为0或属性所有取值相等：返回叶子结点
else：
——for 某一个属性 in 训练样本：#遍历每一个属性
———— for 某属性的某个值 in 某一个属性：
——————根据某属性的某个值，将训练样本划分为两个，计算信息增益
————end for
——end for
——选择最大的信息增益，将这个属性按这个值进行左右分支，并回调这个函数两次

代码模块功能
uniquecounts(rows)：对训练样本的每一个可能的标签结果进行统计，返回dict类型。
entropy(rows)：配合uniquecounts方法，进行信息熵的计算，返回float值。
divideset(rows,column,value)：给定训练样本、第几列、某列中的某个值，效果是将训练样本以某列的某个值一分为二，返回俩个训练样本。
class decisionnode()：初始化结点功能，用于构建树。
buildtree(rows,scoref = entropy)：核心函数，参照算法思想的逻辑构写，递归生成结点。

isinstance() 函数来判断一个对象是否是一个已知的类型，类似 type()。
isinstance() 与 type() 区别：
type() 不会认为子类是一种父类类型，不考虑继承关系。
isinstance() 会认为子类是一种父类类型，考虑继承关系

In [28]:
from math import log
def divideset(rows,column,value):
    #对于给定的集合，指定列数和这列某一个值
    if isinstance(value,int) or isinstance(value,float):
        split_function = lambda row:row[column] >= value
        #fun = lambda  参数入口 ：函数体（默认返回运算结果） ，调用 fun（x，2）

    else:
        split_function = lambda row:row[column]==value
    set1 = [row for row in rows if split_function(row)]
    set2 = [row for row in rows if not split_function(row)]
    # (return) row for row in rows
    return(set1,set2)
#定义节点的属性
class decisionnode:
    def __init__(self,col = -1,value = None, results = None, tb = None,fb = None):
        self.col = col   # col是待检验的判断条件所对应的列索引值
        self.value = value # value对应于为了使结果为True，当前列必须匹配的值
        self.results = results #保存的是针对当前分支的结果，它是一个字典
        self.tb = tb ## desision node,右子树，True对应
        self.fb = fb ## desision node,左子树，False对应
# 对y的各种可能的取值出现的个数进行计数.。其他函数利用该函数来计算数据集和的混杂程度例如{'Basic': 6, 'None': 7, 'Premium': 3}
def uniquecounts(rows):
    results = {}
    for row in rows:
        #计数结果在最后一列
        r = row[len(row)-1]
        if r not in results:
            results[r] = 0
        results[r]+=1
    return results # 返回一个字典
def entropy(rows):
    """
    计算当前集合的信息熵H(D)
    """
    log2 = lambda x:log(x)/log(2)
#     log2 = lambda x:log(2,x)
    results = uniquecounts(rows)
    #开始计算熵的值
    ent = 0.0
    for r in results.keys():
        p = float(results[r])/len(rows)
        ent -=  p*log2(p)
    return ent
# 以递归方式构造树

def buildtree(rows,scoref = entropy):
    if len(rows)==0 : return decisionnode()
    current_score = scoref(rows)

    # 定义一些变量以记录最佳拆分条件
    best_gain = 0.0
    best_criteria = None
    best_sets = None

    column_count = len(rows[0]) - 1
    for col in range(0,column_count): #遍历所有属性列
        #在当前列中生成一个由不同值构成的序列
        column_values = set({})  #用集合存放当前列有哪些不同值
        for row in rows:
            column_values.add(row[col])
#         print(column_values)
        #根据这一列中的每个值，尝试对数据集进行拆分
        for value in column_values:  #某个属性可能的所有值，比如第一列：(direct)、'digg'、 'google'、 'kiwitobes'、 'slashdot'
            (set1,set2) = divideset(rows,col,value)

            # 信息增益
            p = float(len(set1))/len(rows)
            gain = current_score - p*scoref(set1) - (1-p)*scoref(set2)
            if gain>best_gain and len(set1)>0 and len(set2)>0:
                best_gain = gain
                best_criteria = (col,value)  #结点分裂的属性
                best_sets = (set1,set2)

    #创建子分支
    if best_gain>0:
        trueBranch = buildtree(best_sets[0])  #递归调用
        falseBranch = buildtree(best_sets[1])
        return decisionnode(col = best_criteria[0],value = best_criteria[1],
                            tb = trueBranch,fb = falseBranch)
    else:
        print(uniquecounts(rows))
        return decisionnode(results = uniquecounts(rows)) #{'Basic': 6, 'None': 7, 'Premium': 3}
tree = buildtree(data)

{'Decision': 1}
{'Yes': 4}
{'Yes': 1}
{'No': 4}
{'No': 1}
{'Yes': 4}


In [29]:
#绘制决策树
from PIL import Image, ImageDraw

# 获取树的显示宽度，即有多少个叶子节点
def getwidth(tree):
    if tree.tb==None and tree.fb==None: return 1
    return getwidth(tree.tb)+getwidth(tree.fb)

# 获取树的显示深度（高度）
def getdepth(tree):
    if tree.tb==None and tree.fb==None: return 0
    return max(getdepth(tree.tb),getdepth(tree.fb))+1

# 绘制树形图
def drawtree(tree,jpeg='tree.jpg'):
    w=getwidth(tree)*100
    h=getdepth(tree)*100+120

    img=Image.new('RGB',(w,h),(255,255,255))
    draw=ImageDraw.Draw(img)

    drawnode(draw,tree,w/2,20)  #根节点坐标
    img.save(jpeg,'JPEG')

# 迭代画树的节点
def drawnode(draw,tree,x,y):
    if tree.results==None:
        # 得到每个分支的宽度
        w1=getwidth(tree.fb)*100
        w2=getwidth(tree.tb)*100

        # 确定此节点所要占据的总空间
        left=x-(w1+w2)/2
        right=x+(w1+w2)/2

        # 绘制判断条件字符串
        draw.text((x-20,y-10),str(tree.col)+':'+str(tree.value),(0,0,0))

        # 绘制到分支的连线
        draw.line((x,y,left+w1/2,y+100),fill=(255,0,0))
        draw.line((x,y,right-w2/2,y+100),fill=(255,0,0))

        # 绘制分支的节点
        drawnode(draw,tree.fb,left+w1/2,y+100)
        drawnode(draw,tree.tb,right-w2/2,y+100)
    else:
        txt=' \n'.join(['%s:%d'%v for v in tree.results.items()])
        draw.text((x-20,y),txt,(0,0,0))
drawtree(tree,jpeg='tree.jpg')